# Imports

In [25]:
import pandas as pd
import numpy as np
#data visualization
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

pd.options.display.precision = 10

# Umgebungsvariablen

In [6]:
#feature generated 24.03.2019
# path data Domenico
earthquake_daten = '''C:/Users/taacodo4/GITRepos/CAS_PML_Earthquake_pred/data/'''
# path data Stefan
#earthquake_daten = '''C:/studium/studium/CAS_PML/Projekt_Arbeit/earthquake/Daten/earthquake_data/'''
feature_62900_94 = 'Features_62900-94.csv'
feature_41934_94 = 'Features_41934_94.csv'
Features_4194_94 = 'Features_4194_94.csv'

# Feature laden

In [3]:
train_data = pd.read_csv(earthquake_daten+feature_41934_94)

In [4]:
feature_data = train_data.iloc[:,1:len(train_data.columns)-1] # first column is an index column
time_to_failure = np.array(train_data.iloc[:,-1])

In [ ]:
feature_data.head()

# Training / Testdaten vorbereiten

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    feature_data,
    time_to_failure,
    random_state=0,
    test_size=0.25)

In [6]:
scaler = StandardScaler().fit(X_train)
X_train_scaled=scaler.transform(X_train)

C:\Users\chianti\.conda\envs\TensorFlow-GPU-Keras\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\chianti\.conda\envs\TensorFlow-GPU-Keras\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


# Model compilation

In [ ]:
model = Sequential()
model.add(Dense(30, activation='relu', input_dim=94))
model.add(Dense(40, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mae',optimizer='sgd',metrics=['mae', 'acc'],)
#model.compile(loss='mae', optimizer='adam')

model.fit(X_train_scaled, y_train, epochs=100, batch_size=10)

In [21]:
X_test_scaled=scaler.transform(X_test)

C:\Users\taacodo4\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 30)                2850      
_________________________________________________________________
dense_18 (Dense)             (None, 40)                1240      
_________________________________________________________________
dense_19 (Dense)             (None, 30)                1230      
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 31        
Total params: 5,351
Trainable params: 5,351
Non-trainable params: 0
_________________________________________________________________


In [23]:
prediction=model.predict(X_test_scaled)
#Print model report:
print("\nModel Report")
print(model.summary)
print("MAE train data: %.4g" % mean_absolute_error(y_test, prediction))


Model Report
<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x00000123187E60F0>>
MAE train data: 1.774


In [ ]:
prediction=model.predict(X_test_scaled)
#Print model report:
print("\nModel Report")
print("Settings: PCA 20, 30/40/30, 100 epochs, 10 batch size, optimizer sgd, loss function mae")
print(model.summary)
print("MAE train data: %.4g" % mean_absolute_error(y_test, prediction))

# Tuning

Grid search FFN:
https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

Design of NN:
https://playground.tensorflow.org/#activation=tanh&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=4,2&seed=0.19677&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false

Loss function: 
https://isaacchanghau.github.io/post/loss_functions/

## Concept
The neuronal network shall be optimized on following hyperparameter:
* optimization algorithms
* batch size and training epochs
* learning rate and momentum
* network weight initialization
* activation function
* dropout regularization
* number of neurons in the hidden layer

For this purpose the smaller data set of 4k shall be used.

In [21]:
# Function to create model, required for KerasClassifier
def create_model(optimizer='Adam'):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=94, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

## Load data

In [7]:
train_data = pd.read_csv(earthquake_daten+Features_4194_94)

In [8]:
feature_data = train_data.iloc[:,1:len(train_data.columns)-1] # first column is an index column
time_to_failure = np.array(train_data.iloc[:,-1])

## Optimization algorithm

In [16]:
input_variables=feature_data
output_variable=time_to_failure

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1)
grid_result = grid.fit(input_variables, output_variable)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.000477 using {'optimizer': 'RMSprop'}
0.000238 (0.000477) with: {'optimizer': 'SGD'}
0.000477 (0.000584) with: {'optimizer': 'RMSprop'}
0.000477 (0.000584) with: {'optimizer': 'Adagrad'}
0.000000 (0.000000) with: {'optimizer': 'Adadelta'}
0.000477 (0.000584) with: {'optimizer': 'Adam'}
0.000000 (0.000000) with: {'optimizer': 'Adamax'}
0.000477 (0.000584) with: {'optimizer': 'Nadam'}


## Batch size and number of Epoche

In [17]:
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5, n_jobs=-1)
grid_result = grid.fit(input_variables, output_variable)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.000477 using {'batch_size': 10, 'epochs': 100}
0.000238 (0.000477) with: {'batch_size': 10, 'epochs': 10}
0.000238 (0.000477) with: {'batch_size': 10, 'epochs': 50}
0.000477 (0.000584) with: {'batch_size': 10, 'epochs': 100}
0.000238 (0.000477) with: {'batch_size': 20, 'epochs': 10}
0.000000 (0.000000) with: {'batch_size': 20, 'epochs': 50}
0.000000 (0.000000) with: {'batch_size': 20, 'epochs': 100}
0.000000 (0.000000) with: {'batch_size': 40, 'epochs': 10}
0.000477 (0.000584) with: {'batch_size': 40, 'epochs': 50}
0.000238 (0.000477) with: {'batch_size': 40, 'epochs': 100}
0.000000 (0.000000) with: {'batch_size': 60, 'epochs': 10}
0.000000 (0.000000) with: {'batch_size': 60, 'epochs': 50}
0.000238 (0.000477) with: {'batch_size': 60, 'epochs': 100}
0.000238 (0.000477) with: {'batch_size': 80, 'epochs': 10}
0.000238 (0.000477) with: {'batch_size': 80, 'epochs': 50}
0.000238 (0.000477) with: {'batch_size': 80, 'epochs': 100}
0.000477 (0.000584) with: {'batch_size': 100, 'epochs':

## Learning Rate and Momentum

In [28]:
def create_model(learn_rate=0.01):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=94, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    optimizer = Adam(lr=learn_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(learn_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5, n_jobs=-1)
grid_result = grid.fit(input_variables, output_variable)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))